In [1]:
import h5py

In [3]:
import glob

In [7]:
datafile=glob.glob('/Users/nils/CC/afm-cnn/data/raw/features/*')
datafile=datafile[0]

In [8]:
datafile

'/Users/nils/CC/afm-cnn/data/raw/features/cvsift-vlad-256.h5'

In [9]:
keys, features = load_representations(datafile)


In [18]:
keys

array(['1', '10', '100', ..., '997', '998', '999'],
      dtype='<U4')

In [17]:
import pandas as pd
df=pd.DataFrame({'keys':keys,'features':features})
df

Exception: Data must be 1-dimensional

In [13]:
keys.shape

(3299,)

In [12]:
features.shape

(3299, 32768)

In [21]:
kk=[1, 2, 30, 55]
[int(s) for s in [str(k) for k in kk]]

[1, 2, 30, 55]

In [1]:
from svm_param_select2 import *

In [ ]:
keys,features = loa

In [13]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:////Volumes/Macintosh HD/Users/Imperssonator/CC/uhcs/data/microstructures.sqlite')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
db = DBSession()

In [14]:
n_per_class=200
n_repeats=10
C=1
seed = 0
keys, features = load_representations(datafile)


In [15]:
len(keys)

0

In [ ]:

labels = []
for key in keys:
    if '-' in key:
        # deal with cropped micrographs: key -> Micrograph.id-UL
        m_id, quadrant = key.split('-')
    else:
        m_id = key
    m = db.query(Micrograph).filter(Micrograph.micrograph_id == int(m_id)).one()
    labels.append(m.primary_microconstituent)
labels = np.array(labels)

# simplify: get primary microconstituent; throw out martensite
primary_label = np.array([label.split('+')[0] for label in labels])
k = np.array(keys)[primary_label != 'martensite']
l = primary_label[primary_label != 'martensite']
X = features[primary_label != 'martensite']

lb, Xb, sel = select_balanced_dataset(l, X, n_per_class=n_per_class, seed=seed)

cv = StratifiedKFold(n_splits=10, shuffle=True)
# cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1)


In [47]:
score, std, tscore, tstd = cv_loop_linear(lb, Xb, cv, C=C,
                                                      n_repeats=n_repeats)

train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446]
validate score
[0.98333333333333328]
train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446, 0.99259259259259258]
validate score
[0.98333333333333328, 1.0]
train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446, 0.99259259259259258, 0.99444444444444446]
validate score
[0.98333333333333328, 1.0, 1.0]
train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446, 0.99259259259259258, 0.99444444444444446, 0.99629629629629635]
validate score
[0.98333333333333328, 1.0, 1.0, 0.96666666666666667]
train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446, 0.99259259259259258, 0.99444444444444446, 0.99629629629629635, 0.99444444444444446]
validate score
[0.98333333333333328, 1.0, 1.0, 0.96666666666666667, 0.98333333333333328]
train shape
(540, 16384)
test shape
(60, 16384)
train score
[0.99444444444444446, 0.9925925925925

KeyboardInterrupt: 

In [68]:
splits=cv.split(Xb,lb)
for train,test in splits:
    print(lb[test])

['network' 'network' 'network' 'network' 'network' 'network' 'network'
 'network' 'network' 'network' 'network' 'network' 'network' 'network'
 'network' 'network' 'network' 'network' 'network' 'network' 'pearlite'
 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite'
 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite'
 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite'
 'pearlite' 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite'
 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite'
 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite'
 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite' 'spheroidite'
 'spheroidite']
['network' 'network' 'network' 'network' 'network' 'network' 'network'
 'network' 'network' 'network' 'network' 'network' 'network' 'network'
 'network' 'network' 'network' 'network' 'network' 'network' 'pearlite'
 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite' 'pearlite'
 'pearl

In [69]:
np.unique(l)

array(['network', 'pearlite', 'spheroidite'], 
      dtype='<U11')

In [70]:
len(l)

925

In [10]:
from ntsne import ntsne

AttributeError: 'module' object has no attribute 'which'

In [11]:
import shutil

In [17]:
import os
import re
import shutil
import struct
import tempfile
import subprocess
import numpy as np

# vdM's bh_tsne reads and writes from/to these hardcoded paths
# in the directory from which bh_tsne is run
DATAFILE = 'data.dat'
RESULTFILE = 'result.dat'
TSNESOURCE = 'https://github.com/lvdmaaten/bhtsne'

# check to see if bh_tsne in on system PATH or in the CWD;
# if not, clone and build it in ~/.ntsne if necessary
augmented_path = os.getenv('PATH') + ':{}'.format(os.getcwd())
# TSNE = shutil.which('bh_tsne', path=augmented_path)
TSNE=None
if TSNE is None:
    TSNEDIR = os.path.expanduser('~/.ntsne')
    TSNE = os.path.join(TSNEDIR, 'bh_tsne')

In [15]:
augmented_path

'/anaconda/envs/pycv2/bin:/anaconda/envs/pycv2/bin:/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/nils/CC/afm-cnn/scripts'

In [18]:
TSNE

'/Users/nils/.ntsne/bh_tsne'

In [19]:
cwd

NameError: name 'cwd' is not defined

In [20]:
os.getcwd()

'/Users/nils/CC/afm-cnn/scripts'